In [3]:
import os
import pandas as pd
datasets = {}
for dataset in os.listdir("datasets"):
	dataset_path = os.path.join("datasets", dataset)
	dataset_name = dataset.split(".")[0]
	datasets[dataset_name] = pd.read_csv(dataset_path)

/tmp/ipykernel_20870/1770902902.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
datasets['testing_data']

,Cycle,Time Measured(Sec),Voltage Measured(V),Current Measured,Temperature Measured,Capacity(Ah),SampleId
0,0,3690.234,3.277170,-0.006528,34.230853,1.856487,B0005
1,1,3672.344,3.300245,-0.000448,34.392137,1.846327,B0005
2,2,3651.641,3.327451,0.001026,34.232779,1.835349,B0005
3,3,3631.563,3.314182,-0.002234,34.413450,1.835263,B0005
4,4,3629.172,3.305497,0.000009,34.345885,1.834646,B0005
...,...,...,...,...,...,...,...
163,163,2793.390,3.615561,-0.002548,34.075649,1.293464,B0005
164,164,2793.390,3.621191,-0.002460,33.991350,1.288003,B0005
165,165,2792.485,3.619185,0.000478,34.059198,1.287453,B0005
166,166,2802.016,3.595365,-0.002342,34.556514,1.309015,B0005


In [7]:
nasa_battery = pd.read_csv('datasets/B0005.csv')

In [8]:
nasa_battery

,Voltage_measured,Current_measured,Temperature_measured,Current_charge,Voltage_charge,Time,type,start_time,ambient_temp,Capacity,Sense_current,Battery_current,Current_ratio,Battery_impedance,Rectified_impedance,Re,Rct
0,3.873017,-0.001201,24.655358,0.000,0.003,0.000,charge,2008-04-02 13:08:17.000920,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.479394,-4.030268,24.666480,-4.036,1.570,2.532,charge,2008-04-02 13:08:17.000920,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.000588,1.512731,24.675394,1.500,4.726,5.500,charge,2008-04-02 13:08:17.000920,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.012395,1.509063,24.693865,1.500,4.742,8.344,charge,2008-04-02 13:08:17.000920,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.019708,1.511318,24.705069,1.500,4.753,11.125,charge,2008-04-02 13:08:17.000920,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604797,0.236356,-0.003484,23.372048,0.000,0.003,0.000,charge,2008-05-28 11:09:42.000045,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
604798,0.003365,-0.001496,23.369434,0.000,0.003,2.547,charge,2008-05-28 11:09:42.000045,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
604799,4.985137,0.000506,23.386535,0.000,5.002,5.500,charge,2008-05-28 11:09:42.000045,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
604800,4.984720,0.000442,23.386983,-0.002,5.002,8.312,charge,2008-05-28 11:09:42.000045,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
